In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
import feather
import warnings
from IPython.display import display
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
hv = pd.read_feather("hv.feather")

In [4]:
hv.shape

(1745325, 25)

In [5]:
hv.head()

,Unnamed: 0,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,2,INV-24107200133,12/23/2019,2508,Hy-Vee Food Store #1 / Cedar Rapids,"1843 Johnson Avenue, N.W.",Cedar Rapids,52405.0,POINT (-91.697941 41.97447),57.0,LINN,1041100.0,American Dry Gins,370.0,PERNOD RICARD USA,32236,Seagrams Extra Dry Gin,12.0,750.0,6.49,9.74,12.0,116.88,9.00,2.37
1,3,INV-24090200052,12/20/2019,2649,Hy-Vee #3 / Dubuque,400 Locust St,Dubuque,52001.0,POINT (-90.666497 42.49721900000001),31.0,DUBUQUE,1011400.0,Tennessee Whiskies,85.0,Brown Forman Corp.,26826,Jack Daniels Old #7 Black Label,12.0,750.0,15.57,23.36,12.0,280.32,9.00,2.37
2,6,INV-24117700068,12/23/2019,2567,Hy-Vee Drugstore / Davenport,2200 West Kimberly,Davenport,52806.0,POINT (-90.608201 41.560663),82.0,SCOTT,1011200.0,Straight Bourbon Whiskies,65.0,Jim Beam Brands,19066,Jim Beam,12.0,750.0,10.50,15.75,6.0,94.50,4.50,1.18
3,7,INV-24105000057,12/23/2019,2538,Hy-Vee Food Store #3 / Waterloo,1422 Flammang Dr,Waterloo,50702.0,POINT (-92.327917 42.459938),7.0,BLACK HAWK,1011100.0,Blended Whiskies,65.0,Jim Beam Brands,24456,Kessler Blend Whiskey,12.0,750.0,5.51,8.27,3.0,24.81,2.25,0.59
4,12,INV-24117300020,12/23/2019,2614,Hy-Vee #3 Food & Drugstore / Davenport,1823 E Kimberly Rd,Davenport,52807.0,POINT (-90.548919 41.55678100000001),82.0,SCOTT,1011100.0,Blended Whiskies,65.0,Jim Beam Brands,27680,Jim Beam Kentucky Fire Mini,12.0,50.0,5.40,8.10,12.0,97.20,0.60,0.15


# Creating Store category column

In [6]:
hv["Store Category"] = "Supermarket"
hv.loc[hv["Store Name"].str.contains('Food Store'), "Store Category"] = 'Food Store'

In [7]:
hv.loc[hv["Store Name"].str.contains("Wine & Spirits"), "Store Category"] = 'Wine & Spirits'
hv.loc[hv["Store Name"].str.contains("Wine and Spirits"), "Store Category"] = 'Wine & Spirits'

In [8]:
hv.loc[hv["Store Name"].str.contains("Food & Drugstore"), "Store Category"] = "Food & Drugstore"
hv.loc[hv["Store Name"].str.contains("Food and Drug"), "Store Category"] = "Food & Drugstore"
hv.loc[hv["Store Name"].str.contains("Food and Drugstore"), "Store Category"] = "Food & Drugstore"

In [9]:
hv.loc[hv["Store Name"].str.contains("Gas"), "Store Category"] = "Gas"

In [10]:
hv.loc[hv["Store Name"].str.contains("Fast & Fresh"), "Store Category"] = "Fast & Fresh"
hv.loc[hv["Store Name"].str.contains("Fast and Fresh"), "Store Category"] = "Fast & Fresh"

In [11]:
hv.loc[hv["Store Name"].str.contains("Drugstore"), "Store Category"] = "Drugstore"

In [12]:
floattoint = ["Zip Code", "County Number", "Category", "Vendor Number", "Item Number", "Pack", "Bottles Sold"]

for i in floattoint:
    hv[i] = hv[i].astype("Int64")

In [13]:
hv = hv.drop(columns=["Unnamed: 0"])

# Missing Values

In [14]:
null_counts = hv.isnull().sum()
null_counts

Invoice/Item Number           0
Date                          0
Store Number                  0
Store Name                    0
Address                    1503
City                       1503
Zip Code                   1503
Store Location           266054
County Number              1503
County                     1503
Category                    418
Category Name               420
Vendor Number                 1
Vendor Name                   1
Item Number                   0
Item Description              0
Pack                          1
Bottle Volume (ml)            1
State Bottle Cost             1
State Bottle Retail           1
Bottles Sold                  1
Sale (Dollars)                1
Volume Sold (Liters)          1
Volume Sold (Gallons)         1
Store Category                0
dtype: int64

## Missing Category and Category Name

In [15]:
hv.loc[hv["Category"].isna()].head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
19,INV-24110700158,12/23/2019,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14th St,Des Moines,50320,POINT (-93.596754 41.554101),77,POLK,<NA>,None,305,Mhw LTD,80024,Blue Chair Bay Banana Rum Cream,12,750.0,9.42,14.13,12,169.56,9.00,2.37,Supermarket
232,INV-24111100086,12/23/2019,2527,Hy-Vee Food Store #5 / Des Moines,1107 SE Army Post Rd,Des Moines,50315,POINT (-93.602372 41.526289),77,POLK,<NA>,None,305,Mhw LTD,80024,Blue Chair Bay Banana Rum Cream,12,750.0,9.42,14.13,12,169.56,9.00,2.37,Food Store
927,INV-24105000123,12/23/2019,2538,Hy-Vee Food Store #3 / Waterloo,1422 Flammang Dr,Waterloo,50702,POINT (-92.327917 42.459938),7,BLACK HAWK,<NA>,None,305,Mhw LTD,80024,Blue Chair Bay Banana Rum Cream,12,750.0,9.42,14.13,3,42.39,2.25,0.59,Food Store
982,INV-24109000198,12/23/2019,2572,Hy-Vee Food Store / Cedar Falls,6301 University,Cedar Falls,50613,POINT (-92.435236 42.512789),7,BLACK HAWK,<NA>,None,305,Mhw LTD,80024,Blue Chair Bay Banana Rum Cream,12,750.0,9.42,14.13,3,42.39,2.25,0.59,Food Store
1583,INV-24115000095,12/23/2019,2619,Hy-Vee Wine and Spirits / WDM,1725 74th St,West Des Moines,50266,POINT (-93.808855 41.598515),77,POLK,<NA>,None,305,Mhw LTD,80024,Blue Chair Bay Banana Rum Cream,12,750.0,9.42,14.13,3,42.39,2.25,0.59,Wine & Spirits


In [16]:
missing_category_item_numbers = hv[hv["Category"].isna()]["Item Number"].dropna().unique()

In [ ]:
missing_category_name_item_numbers = hv[hv["Category Name"].isna()]["Item Number"].dropna().unique()

In [17]:
for item_number in missing_category_item_numbers:
    # Grab mode for `Category` using `Item Number`
    category_from_mode = hv.loc[(hv["Item Number"] == item_number)]["Category"].mode()

    # For now skipping none mode values
    if len(category_from_mode) == 0:
        continue
        
    # If it's not empty grab first value (mode)
    category_from_mode = category_from_mode[0]
    
    # Find where `Category` is missing ("N/A" or "None")
    #    Then replace with new `Category` from mode
    hv.loc[hv['Item Number'] == item_number, 'Category'] = hv.loc[hv['Item Number'] == item_number, 'Category'].fillna(category_from_mode)
    
for item_number in missing_category_name_item_numbers:
    # Grab mode for `Category Name` using `Item Number`
    category_name_from_mode = hv.loc[(hv["Item Number"] == item_number)]["Category Name"].mode()

    # For now skipping none mode values
    if len(category_name_from_mode) == 0:
        continue
        
    # If it's not empty grab first value (mode)
    category_name_from_mode = category_name_from_mode[0]
    
    # Find where `Category Name` is missing ("N/A" or "None")
    #    Then replace with new `Category Name` from mode
    hv.loc[hv['Item Number'] == item_number, 'Category Name'] = hv.loc[hv['Item Number'] == item_number, 'Category Name'].fillna(category_name_from_mode)

In [18]:
# Find all item numbers of null Categories
new_missing_category_item_numbers = hv[hv['Category'].isna()]['Item Number'].unique()
# Find all item numbers of null Categories Names
new_missing_category_name_item_numbers = hv[hv['Category Name'].isna()]['Item Number'].unique()

# New missing items should only have null modes
# meaning all same items in dataset are null (we can't set them to similar items)
for item_number in new_missing_category_item_numbers:
    category_from_mode = hv.loc[(hv["Item Number"] == item_number)]["Category"].mode()
    display(category_from_mode)
    
for item_number in new_missing_category_name_item_numbers:
    category_name_from_mode = hv.loc[(hv["Item Number"] == item_number)]["Category Name"].mode()
    display(category_name_from_mode)

Series([], dtype: Int64)

Series([], dtype: Int64)

Series([], dtype: Int64)

Series([], dtype: Int64)

Series([], dtype: Int64)

...

Series([], dtype: object)

Series([], dtype: object)

Series([], dtype: object)

Series([], dtype: object)

Series([], dtype: object)

In [19]:
hv.loc[hv["Category"].isna()]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
54848,INV-24573900003,01/15/2020,2625,Hy-Vee Wine & Spirits #2 / Davenport,3301 W Kimberly Rd,Davenport,52804,None,82,SCOTT,<NA>,None,205,E & J Gallo Winery,994771,SOOH Diplomatico Reserva Exclusiva,6,750.0,19.33,29.00,6,174.00,4.50,1.18,Wine & Spirits
1172920,INV-20382200001,07/03/2019,2648,Hy-Vee #4 / WDM,555 S 51st St,West Des Moines,50265,POINT (-93.773557 41.561197),77,POLK,<NA>,None,322,Prestige Wine & Spirits Group,994811,Athena Ouzo Rum,12,750.0,6.06,9.09,12,109.08,9.00,2.37,Supermarket
1291094,INV-25817300011,03/11/2020,2673,Hy-Vee # 6/ Des Moines,420 Court Ave,Des Moines,50309,POINT (-93.62245400000002 41.584842),77,POLK,<NA>,None,205,E & J Gallo Winery,994771,Diplomatico Reserva Exclusiva,6,750.0,19.33,29.00,6,174.00,4.50,1.18,Supermarket
1292514,INV-25844800003,03/12/2020,2626,Hy-Vee Drugstore / University / DSM,4100 University Ave,Des Moines,50311,POINT (-93.673222 41.600361),77,POLK,<NA>,None,205,E & J Gallo Winery,994771,Diplomatico Reserva Exclusiva,6,750.0,19.33,29.00,6,174.00,4.50,1.18,Drugstore
1295098,INV-25824400019,03/12/2020,2675,Hy-Vee #2 / Coralville,3285 Crosspark Rd,Coralville,52241,POINT (-91.60627100000002 41.721438),52,JOHNSON,<NA>,None,205,E & J Gallo Winery,994771,Diplomatico Reserva Exclusiva,6,750.0,19.33,29.00,6,174.00,4.50,1.18,Supermarket
1436349,INV-26611600001,04/16/2020,2515,Hy-Vee Food Store #1 / Mason City,2400 4th St SW,Mason City,50401,POINT (-93.235588 43.148451),17,CERRO GORD,<NA>,None,65,Jim Beam Brands,973508,Dekuyper Mixologist Ginger,12,750.0,7.36,11.04,24,264.96,18.00,4.75,Food Store
1483107,INV-01622300001,11/16/2016,2556,Hy-Vee Wine and Spirits / Estherville,None,None,<NA>,None,<NA>,None,<NA>,None,260,DIAGEO AMERICAS,115,Crown Royal w/Glass & Coaster,6,750.0,15.07,22.61,6,135.66,4.50,1.19,Wine & Spirits
1562201,INV-27171500042,05/11/2020,2662,Hy-Vee Wine & Spirits / Muscatine,"522 Mulberry, Suite A",Muscatine,52761,POINT (-91.04437 41.426615),70,MUSCATINE,<NA>,None,125,CEDAR RIDGE VINEYARDS LL,27849,Cedar Ridge Reserve Bourbon,6,750.0,25.47,38.21,1,38.21,0.75,0.19,Wine & Spirits
1580086,INV-27345600003,05/19/2020,2538,Hy-Vee Food Store #3 / Waterloo,1422 Flammang Dr,Waterloo,50702,POINT (-92.327917 42.459938),7,BLACK HAWK,<NA>,None,125,CEDAR RIDGE VINEYARDS LL,926786,Cedar Ridge Private Cask Selection Malted Rye ...,6,750.0,26.84,40.26,204,8213.04,153.00,40.41,Food Store


In [20]:
null_counts = hv.isnull().sum()
null_counts

Invoice/Item Number           0
Date                          0
Store Number                  0
Store Name                    0
Address                    1503
City                       1503
Zip Code                   1503
Store Location           266054
County Number              1503
County                     1503
Category                      9
Category Name                11
Vendor Number                 1
Vendor Name                   1
Item Number                   0
Item Description              0
Pack                          1
Bottle Volume (ml)            1
State Bottle Cost             1
State Bottle Retail           1
Bottles Sold                  1
Sale (Dollars)                1
Volume Sold (Liters)          1
Volume Sold (Gallons)         1
Store Category                0
dtype: int64

### Diplomatico Reserva Exclusiva and SOOH Diplomatico Reserva Exclusiva

In [21]:
hv.loc[(hv["Item Description"] == "Diplomatico Reserva Exclusiva") & (hv["Category Name"].notna())][["Item Description", "Category", "Category Name"]].head()

,Item Description,Category,Category Name
92170,Diplomatico Reserva Exclusiva,1062300,Aged Dark Rum
93315,Diplomatico Reserva Exclusiva,1062300,Aged Dark Rum
116260,Diplomatico Reserva Exclusiva,1062300,Aged Dark Rum
124886,Diplomatico Reserva Exclusiva,1062300,Aged Dark Rum
129055,Diplomatico Reserva Exclusiva,1062300,Aged Dark Rum


In [22]:
#only one transaction for SOOH Diplomatico Reserva Exclusiva, 
# will give category and cat name same as Diplomatico Reserva Exclusiva

hv.loc[(hv["Item Description"] == "SOOH Diplomatico Reserva Exclusiva")]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
54848,INV-24573900003,01/15/2020,2625,Hy-Vee Wine & Spirits #2 / Davenport,3301 W Kimberly Rd,Davenport,52804,None,82,SCOTT,<NA>,None,205,E & J Gallo Winery,994771,SOOH Diplomatico Reserva Exclusiva,6,750.0,19.33,29.0,6,174.0,4.5,1.18,Wine & Spirits


In [23]:
hv.loc[hv['Item Description'] == "Diplomatico Reserva Exclusiva", 'Category'] = 1062300
hv.loc[hv['Item Description'] == "Diplomatico Reserva Exclusiva", 'Category Name'] = "Aged Dark Rum"

hv.loc[(hv["Item Description"] == "Diplomatico Reserva Exclusiva") & (hv["Category"].isna())][["Item Description","Category", "Category Name"]]

,Item Description,Category,Category Name


In [24]:
hv.loc[hv['Item Description'] == "SOOH Diplomatico Reserva Exclusiva", 'Category'] = 1062300
hv.loc[hv['Item Description'] == "SOOH Diplomatico Reserva Exclusiva", 'Category Name'] = "Aged Dark Rum"

hv.loc[(hv["Item Description"] == "SOOH Diplomatico Reserva Exclusiva") & (hv["Category"].isna())][["Item Description","Category", "Category Name"]]

,Item Description,Category,Category Name


### Athena Ouzo Rum

In [25]:
hv.loc[(hv["Item Description"] == "Athena Ouzo Rum")]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
1172920,INV-20382200001,07/03/2019,2648,Hy-Vee #4 / WDM,555 S 51st St,West Des Moines,50265,POINT (-93.773557 41.561197),77,POLK,<NA>,None,322,Prestige Wine & Spirits Group,994811,Athena Ouzo Rum,12,750.0,6.06,9.09,12,109.08,9.0,2.37,Supermarket


In [26]:
hv.loc[(hv["Item Number"] == 994811)]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
1172920,INV-20382200001,07/03/2019,2648,Hy-Vee #4 / WDM,555 S 51st St,West Des Moines,50265,POINT (-93.773557 41.561197),77,POLK,<NA>,None,322,Prestige Wine & Spirits Group,994811,Athena Ouzo Rum,12,750.0,6.06,9.09,12,109.08,9.0,2.37,Supermarket


#### Athena Ouzo Rum is missing a category name and number
- Filtering item description by those that contain "Rum" and also filter by County POLK
- Looking online, Athena Ouzo spirit style is Herbal & Spice rum
- Setting Category Name to Spiced Rum and using the most common Category in POLK county for Rum Category

In [27]:
hv.loc[(hv["Item Description"] == "Athena Ouzo Rum") & (hv["Category Name"].notna())][["Category", "Category Name"]].head()

,Category,Category Name


In [28]:
hv[(hv['Item Description'].str.contains('Rum', regex=False)) & (hv["County"] == "POLK")].head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
19,INV-24110700158,12/23/2019,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14th St,Des Moines,50320,POINT (-93.596754 41.554101),77,POLK,1062500,Flavored Rum,305,Mhw LTD,80024,Blue Chair Bay Banana Rum Cream,12,750.0,9.42,14.13,12,169.56,9.0,2.37,Supermarket
69,INV-24110700091,12/23/2019,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14th St,Des Moines,50320,POINT (-93.596754 41.554101),77,POLK,1062200,White Rum,380,Phillips Beverage,45419,Phillips White Rum,12,1000.0,4.45,6.68,36,240.48,36.0,9.51,Supermarket
232,INV-24111100086,12/23/2019,2527,Hy-Vee Food Store #5 / Des Moines,1107 SE Army Post Rd,Des Moines,50315,POINT (-93.602372 41.526289),77,POLK,1062500,Flavored Rum,305,Mhw LTD,80024,Blue Chair Bay Banana Rum Cream,12,750.0,9.42,14.13,12,169.56,9.0,2.37,Food Store
413,INV-24110400015,12/23/2019,2680,Hy-Vee Fulfillment Center,"3303 86th Street, Suite B",Urbandale,50322,POINT (-93.736458 41.624917),77,POLK,1062400,Spiced Rum,259,Heaven Hill Brands,43028,Admiral Nelson Spiced Rum,6,1750.0,11.55,17.33,6,103.98,10.5,2.77,Supermarket
523,INV-24115000015,12/23/2019,2619,Hy-Vee Wine and Spirits / WDM,1725 74th St,West Des Moines,50266,POINT (-93.808855 41.598515),77,POLK,1081200,Cream Liqueurs,305,Mhw LTD,73051,Rumchata Mini,12,50.0,8.87,13.31,2,26.62,0.1,0.02,Wine & Spirits


In [29]:
hv.loc[hv['Item Description'] == "Athena Ouzo Rum", 'Category'] = 1062500
hv.loc[hv['Item Description'] == "Athena Ouzo Rum", 'Category Name'] = "Spiced Rum"

### White Dog Straight Rye HA and White Dog Wheat HA

In [30]:
hv.loc[hv["Item Description"] == "White Dog Straight Rye HA", "Category Name"] = "American Distilled Spirit Specialty"
hv.loc[hv["Item Description"] == "White Dog Wheat HA", "Category Name"] = "American Distilled Spirit Specialty"

### Dekuyper Mixologist Ginger

In [31]:
hv.loc[(hv["Item Description"] == "Dekuyper Mixologist Ginger")]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
1436349,INV-26611600001,04/16/2020,2515,Hy-Vee Food Store #1 / Mason City,2400 4th St SW,Mason City,50401,POINT (-93.235588 43.148451),17,CERRO GORD,<NA>,None,65,Jim Beam Brands,973508,Dekuyper Mixologist Ginger,12,750.0,7.36,11.04,24,264.96,18.0,4.75,Food Store


In [32]:
hv.loc[(hv["Item Number"] == 973508)]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
1436349,INV-26611600001,04/16/2020,2515,Hy-Vee Food Store #1 / Mason City,2400 4th St SW,Mason City,50401,POINT (-93.235588 43.148451),17,CERRO GORD,<NA>,None,65,Jim Beam Brands,973508,Dekuyper Mixologist Ginger,12,750.0,7.36,11.04,24,264.96,18.0,4.75,Food Store


In [33]:
hv[(hv['Item Description'].str.contains('Dekuyper', regex=False)) & (hv["Store Name"] == "Hy-Vee Food Store #1 / Mason City")][["Category", "Category Name"]].mode()

,Category,Category Name
0,1081400,American Schnapps


In [34]:
hv.loc[hv["Item Description"] == "Dekuyper Mixologist Ginger", "Category Name"] = "American Schnapps"
hv.loc[hv["Item Description"] == "Dekuyper Mixologist Ginger", "Category"] = 1081400

### Crown Royal w/Glass & Coaster

In [35]:
hv.loc[(hv["Item Description"] == "Crown Royal w/Glass & Coaster")]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
1483107,INV-01622300001,11/16/2016,2556,Hy-Vee Wine and Spirits / Estherville,None,None,<NA>,None,<NA>,None,<NA>,None,260,DIAGEO AMERICAS,115,Crown Royal w/Glass & Coaster,6,750.0,15.07,22.61,6,135.66,4.5,1.19,Wine & Spirits


In [36]:
hv[(hv['Item Description'].str.contains('w/Glass', regex=False))]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
1297,INV-24073000109,12/20/2019,2624,Hy-Vee #2 / Dubuque,2395 NW Arterial Rd,Dubuque,52002,POINT (-90.741064 42.516633),31,DUBUQUE,1701100,Temporary & Specialty Packages,55,SAZERAC NORTH AMERICA,4055,Caravella Limoncello w/Glass,6,750.0,11.25,16.88,6,101.28,4.50,1.18,Supermarket
25961,INV-24323500011,01/03/2020,2626,Hy-Vee Drugstore / University / DSM,4100 University Ave,Des Moines,50311,POINT (-93.673222 41.600361),77,POLK,1701100,Temporary & Specialty Packages,85,Brown Forman Corp.,470,Jack Daniels w/Glass,6,750.0,15.57,23.36,24,560.64,18.00,4.75,Drugstore
35660,INV-24399000028,01/07/2020,2594,Hy-Vee Food Store / Sioux City,4500 Sergeant Road,Sioux City,51106,POINT (-96.346969 42.447396),97,WOODBURY,1701100,Temporary & Specialty Packages,85,Brown Forman Corp.,499,El Jimador Reposado w/Glass,6,750.0,12.17,18.26,12,219.12,9.00,2.37,Food Store
38450,INV-24459100130,01/09/2020,2515,Hy-Vee Food Store #1 / Mason City,2400 4th St SW,Mason City,50401,POINT (-93.235588 43.148451),17,CERRO GORD,1701100,Temporary & Specialty Packages,65,Jim Beam Brands,4043,Hornitos Plata w/Glasses,6,750.0,13.50,20.25,3,60.75,2.25,0.59,Food Store
39592,INV-24462700009,01/09/2020,2636,Hy-Vee Wine and Spirits / Hubbell,2310 Hubbell Ave,Des Moines,50317,POINT (-93.573845 41.604312),77,POLK,1701100,Temporary & Specialty Packages,85,Brown Forman Corp.,470,Jack Daniels w/Glass,6,750.0,15.57,23.36,12,280.32,9.00,2.37,Wine & Spirits
42715,INV-24422600038,01/08/2020,2648,Hy-Vee #4 / WDM,555 S 51st St,West Des Moines,50265,POINT (-93.773557 41.561197),77,POLK,1701100,Temporary & Specialty Packages,85,Brown Forman Corp.,470,Jack Daniels w/Glass,6,750.0,15.57,23.36,120,2803.20,90.00,23.77,Supermarket
45706,INV-24494900002,01/10/2020,2591,Hy-Vee Wine and Spirits / Atlantic,1602 E 7th St,Atlantic,50022,POINT (-94.990031 41.403856),15,CASS,1701100,Temporary & Specialty Packages,85,Brown Forman Corp.,470,Jack Daniels w/Glass,6,750.0,15.57,23.36,48,1121.28,36.00,9.51,Wine & Spirits
49566,INV-24565700131,01/15/2020,2620,Hy-Vee / Windsor Heights,7101 University Ave,Windsor Heights,50311,None,77,Polk,1701100,Temporary & Specialty Packages,259,Heaven Hill Brands,100508,Carolans Irish Cream w/Glass,6,750.0,9.00,13.50,6,73.80,4.50,1.18,Supermarket
49802,INV-24565700132,01/15/2020,2620,Hy-Vee / Windsor Heights,7101 University Ave,Windsor Heights,50311,None,77,Polk,1701100,Temporary & Specialty Packages,259,Heaven Hill Brands,100508,Carolans Irish Cream w/Glass,6,750.0,9.00,13.50,2,24.60,1.50,0.39,Supermarket
54043,INV-24567100005,01/15/2020,2517,Hy-Vee Food Store #1 / Newton,1501 First Avenue East,Newton,50208,POINT (-93.034296 41.69916),50,JASPER,1701100,Temporary & Specialty Packages,85,Brown Forman Corp.,470,Jack Daniels w/Glass,6,750.0,15.57,23.36,36,840.96,27.00,7.13,Food Store


In [37]:
hv.loc[hv["Item Description"] == "Crown Royal w/Glass & Coaster", "Category Name"] = "Temporary & Specialty Packages"
hv.loc[hv["Item Description"] == "Crown Royal w/Glass & Coaster", "Category"] = 1700000

### Cedar Ridge

In [38]:
hv.loc[(hv["Item Description"] == "Cedar Ridge Reserve Bourbon")]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
1562201,INV-27171500042,05/11/2020,2662,Hy-Vee Wine & Spirits / Muscatine,"522 Mulberry, Suite A",Muscatine,52761,POINT (-91.04437 41.426615),70,MUSCATINE,<NA>,None,125,CEDAR RIDGE VINEYARDS LL,27849,Cedar Ridge Reserve Bourbon,6,750.0,25.47,38.21,1,38.21,0.75,0.19,Wine & Spirits


In [39]:
hv[(hv['Item Description'].str.contains('Cedar Ridge', regex=False))].head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
102,INV-24115200018,12/23/2019,2630,Hy-Vee Drugstore #2 / WDM,1010 60th St,West Des Moines,50266,POINT (-93.790534 41.584979),77,POLK,1011600,Straight Rye Whiskies,125,CEDAR RIDGE VINEYARDS LL,27014,Cedar Ridge Malted Rye,6,750.0,19.67,29.51,6,177.06,4.5,1.18,Drugstore
169,INV-24073800077,12/20/2019,2524,Hy-Vee Food Store / Dubuque,3500 Dodge St,Dubuque,52001,None,31,DUBUQUE,1012300,Single Malt Scotch,125,CEDAR RIDGE VINEYARDS LL,27476,Cedar Ridge Single Malt Whiskey,6,750.0,25.00,37.50,6,225.00,4.5,1.18,Food Store
441,INV-24073800035,12/20/2019,2524,Hy-Vee Food Store / Dubuque,3500 Dodge St,Dubuque,52001,None,31,DUBUQUE,1011200,Straight Bourbon Whiskies,125,CEDAR RIDGE VINEYARDS LL,17206,Cedar Ridge Bourbon,6,750.0,18.10,27.15,12,325.80,9.0,2.37,Food Store
558,INV-24081400078,12/20/2019,2551,Hy-Vee Food Store / Chariton,2001 West Court,Chariton,50049,None,59,LUCAS,1011600,Straight Rye Whiskies,125,CEDAR RIDGE VINEYARDS LL,27014,Cedar Ridge Malted Rye,6,750.0,19.67,29.51,2,59.02,1.5,0.39,Food Store
566,INV-24115000113,12/23/2019,2619,Hy-Vee Wine and Spirits / WDM,1725 74th St,West Des Moines,50266,POINT (-93.808855 41.598515),77,POLK,1011200,Straight Bourbon Whiskies,125,CEDAR RIDGE VINEYARDS LL,17206,Cedar Ridge Bourbon,6,750.0,18.10,27.15,6,162.90,4.5,1.18,Wine & Spirits


In [40]:
hv.loc[hv["Item Description"] == "Cedar Ridge Reserve Bourbon", "Category Name"] = "Straight Bourbon Whiskies"
hv.loc[hv["Item Description"] == "Cedar Ridge Reserve Bourbon", "Category"] = 17206

In [41]:
hv.loc[hv["Category"].isna()].head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
1580086,INV-27345600003,05/19/2020,2538,Hy-Vee Food Store #3 / Waterloo,1422 Flammang Dr,Waterloo,50702,POINT (-92.327917 42.459938),7,BLACK HAWK,<NA>,None,125,CEDAR RIDGE VINEYARDS LL,926786,Cedar Ridge Private Cask Selection Malted Rye ...,6,750.0,26.84,40.26,204,8213.04,153.0,40.41,Food Store


In [42]:
hv[(hv['Item Description'].str.contains('Cedar Ridge Private Cask', regex=False))].head(10)

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
638964,INV-36984500001,05/26/2021,2647,Hy-Vee #7 / Cedar Rapids,5050 Edgewood Rd,Cedar Rapids,52411,POINT (-91.701581 42.030129),57,LINN,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,928271,Cedar Ridge Private Cask Iowa Bourbon Selectio...,6,750.0,23.00,34.50,216,7452.00,162.0,42.79,Supermarket
1172929,INV-20346900001,07/02/2019,2544,Hy-Vee Food Store / Marshalltown,802 South Center,Marshalltown,50158,POINT (-92.912817 42.039205),64,MARSHALL,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,926685,Cedar Ridge Private Cask Wheat Whiskey Selecti...,6,750.0,26.84,40.26,222,8937.72,166.5,43.98,Food Store
1220984,INV-28427700001,07/01/2020,2512,Hy-Vee Wine and Spirits / Iowa City,1720 Waterfront Dr,Iowa City,52240,POINT (-91.53046300000001 41.642764),52,JOHNSON,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,928270,Cedar Ridge Private Cask Iowa Bourbon Selectio...,6,750.0,21.00,31.50,252,7938.00,189.0,49.92,Wine & Spirits
1427527,INV-26574700001,04/15/2020,2648,Hy-Vee #4 / WDM,555 S 51st St,West Des Moines,50265,POINT (-93.773557 41.561197),77,POLK,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,928270,Cedar Ridge Private Cask Iowa Bourbon Selectio...,6,750.0,21.00,31.50,252,7938.00,189.0,49.92,Supermarket
1580086,INV-27345600003,05/19/2020,2538,Hy-Vee Food Store #3 / Waterloo,1422 Flammang Dr,Waterloo,50702,POINT (-92.327917 42.459938),7,BLACK HAWK,<NA>,None,125,CEDAR RIDGE VINEYARDS LL,926786,Cedar Ridge Private Cask Selection Malted Rye ...,6,750.0,26.84,40.26,204,8213.04,153.0,40.41,Food Store
1584546,INV-27390800001,05/20/2020,2590,Hy-Vee Food Store #5 / Cedar Rapids,3235 Oakland Road NE,Cedar Rapids,52402,POINT (-91.658105 42.010971),57,LINN,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,928271,Cedar Ridge Private Cask Iowa Bourbon Selectio...,6,750.0,23.00,34.50,204,7038.00,153.0,40.41,Food Store
1674650,INV-28239000001,06/24/2020,2670,Hy-Vee Food Store / Coralville,2004 8th St,Coralville,52241,POINT (-91.591868 41.682337000000004),52,JOHNSON,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,928270,Cedar Ridge Private Cask Iowa Bourbon Selectio...,6,750.0,21.00,31.50,264,8316.00,198.0,52.30,Food Store


In [43]:
hv[(hv['Item Description'].str.contains('Cedar Ridge Private Cask', regex=False))].head(10)

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
638964,INV-36984500001,05/26/2021,2647,Hy-Vee #7 / Cedar Rapids,5050 Edgewood Rd,Cedar Rapids,52411,POINT (-91.701581 42.030129),57,LINN,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,928271,Cedar Ridge Private Cask Iowa Bourbon Selectio...,6,750.0,23.00,34.50,216,7452.00,162.0,42.79,Supermarket
1172929,INV-20346900001,07/02/2019,2544,Hy-Vee Food Store / Marshalltown,802 South Center,Marshalltown,50158,POINT (-92.912817 42.039205),64,MARSHALL,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,926685,Cedar Ridge Private Cask Wheat Whiskey Selecti...,6,750.0,26.84,40.26,222,8937.72,166.5,43.98,Food Store
1220984,INV-28427700001,07/01/2020,2512,Hy-Vee Wine and Spirits / Iowa City,1720 Waterfront Dr,Iowa City,52240,POINT (-91.53046300000001 41.642764),52,JOHNSON,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,928270,Cedar Ridge Private Cask Iowa Bourbon Selectio...,6,750.0,21.00,31.50,252,7938.00,189.0,49.92,Wine & Spirits
1427527,INV-26574700001,04/15/2020,2648,Hy-Vee #4 / WDM,555 S 51st St,West Des Moines,50265,POINT (-93.773557 41.561197),77,POLK,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,928270,Cedar Ridge Private Cask Iowa Bourbon Selectio...,6,750.0,21.00,31.50,252,7938.00,189.0,49.92,Supermarket
1580086,INV-27345600003,05/19/2020,2538,Hy-Vee Food Store #3 / Waterloo,1422 Flammang Dr,Waterloo,50702,POINT (-92.327917 42.459938),7,BLACK HAWK,<NA>,None,125,CEDAR RIDGE VINEYARDS LL,926786,Cedar Ridge Private Cask Selection Malted Rye ...,6,750.0,26.84,40.26,204,8213.04,153.0,40.41,Food Store
1584546,INV-27390800001,05/20/2020,2590,Hy-Vee Food Store #5 / Cedar Rapids,3235 Oakland Road NE,Cedar Rapids,52402,POINT (-91.658105 42.010971),57,LINN,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,928271,Cedar Ridge Private Cask Iowa Bourbon Selectio...,6,750.0,23.00,34.50,204,7038.00,153.0,40.41,Food Store
1674650,INV-28239000001,06/24/2020,2670,Hy-Vee Food Store / Coralville,2004 8th St,Coralville,52241,POINT (-91.591868 41.682337000000004),52,JOHNSON,1011100,Blended Whiskies,125,CEDAR RIDGE VINEYARDS LL,928270,Cedar Ridge Private Cask Iowa Bourbon Selectio...,6,750.0,21.00,31.50,264,8316.00,198.0,52.30,Food Store


In [44]:
hv[(hv['Item Description'].str.contains('Malted Rye', regex=False))].head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
102,INV-24115200018,12/23/2019,2630,Hy-Vee Drugstore #2 / WDM,1010 60th St,West Des Moines,50266,POINT (-93.790534 41.584979),77,POLK,1011600,Straight Rye Whiskies,125,CEDAR RIDGE VINEYARDS LL,27014,Cedar Ridge Malted Rye,6,750.0,19.67,29.51,6,177.06,4.5,1.18,Drugstore
558,INV-24081400078,12/20/2019,2551,Hy-Vee Food Store / Chariton,2001 West Court,Chariton,50049,None,59,LUCAS,1011600,Straight Rye Whiskies,125,CEDAR RIDGE VINEYARDS LL,27014,Cedar Ridge Malted Rye,6,750.0,19.67,29.51,2,59.02,1.5,0.39,Food Store
4952,INV-24147600048,12/24/2019,2513,Hy-Vee Food Store #2 / Iowa City,812 S 1st Ave,Iowa City,52240,None,52,JOHNSON,1011600,Straight Rye Whiskies,125,CEDAR RIDGE VINEYARDS LL,27014,Cedar Ridge Malted Rye,6,750.0,19.67,29.51,6,177.06,4.5,1.18,Food Store
7049,INV-24145400019,12/24/2019,2575,Hy-Vee Food Store #1 / Waterloo,2834 Ansborough Ave,Waterloo,50702,POINT (-92.37662400000002 42.471767),7,BLACK HAWK,1011600,Straight Rye Whiskies,125,CEDAR RIDGE VINEYARDS LL,27014,Cedar Ridge Malted Rye,6,750.0,19.67,29.51,6,177.06,4.5,1.18,Food Store
8202,INV-24159000089,12/26/2019,2647,Hy-Vee #7 / Cedar Rapids,5050 Edgewood Rd,Cedar Rapids,52411,POINT (-91.701581 42.030129),57,LINN,1011600,Straight Rye Whiskies,125,CEDAR RIDGE VINEYARDS LL,26770,Cedar Ridge Malted Rye,12,200.0,6.00,9.00,3,27.00,0.6,0.15,Supermarket


In [45]:
pd.set_option('max_colwidth', 800)

In [46]:
hv[(hv['Category'].isnull())]["Item Description"]

1580086    Cedar Ridge Private Cask Selection Malted Rye 115prf
Name: Item Description, dtype: object

In [47]:
hv.loc[hv["Item Description"] == "Cedar Ridge Private Cask Selection Malted Rye 115prf", "Category Name"] = "Cedar Ridge Malted Rye"
hv.loc[hv["Item Description"] == "Cedar Ridge Private Cask Selection Malted Rye 115prf", "Category"] = 27014

In [48]:
null_counts = hv.isnull().sum()
null_counts

Invoice/Item Number           0
Date                          0
Store Number                  0
Store Name                    0
Address                    1503
City                       1503
Zip Code                   1503
Store Location           266054
County Number              1503
County                     1503
Category                      0
Category Name                 0
Vendor Number                 1
Vendor Name                   1
Item Number                   0
Item Description              0
Pack                          1
Bottle Volume (ml)            1
State Bottle Cost             1
State Bottle Retail           1
Bottles Sold                  1
Sale (Dollars)                1
Volume Sold (Liters)          1
Volume Sold (Gallons)         1
Store Category                0
dtype: int64

## Vendor Number

In [49]:
hv[hv['Vendor Number'].isna()]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
269636,INV-33728600005,01/25/2021,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14th St,Des Moines,50320,POINT (-93.596754 41.554101),77,POLK,1022200,100% Agave Tequila,<NA>,None,988100,Monte Alban Silver,12,750.0,12.98,15.56,12,186.72,9.0,2.37,Supermarket


In [50]:
#checking who sells other products with that item number
hv[hv['Item Number'] == 988100]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
269636,INV-33728600005,01/25/2021,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14th St,Des Moines,50320,POINT (-93.596754 41.554101),77,POLK,1022200,100% Agave Tequila,<NA>,None,988100,Monte Alban Silver,12,750.0,12.98,15.56,12,186.72,9.0,2.37,Supermarket
343546,INV-41510400006,11/01/2021,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14th St,Des Moines,50320,POINT (-93.596754 41.554101),77,POLK,1022200,100% Agave Tequila,421,SAZERAC COMPANY INC,988100,Monte Alban Silver,12,750.0,10.37,15.56,12,186.72,9.0,2.37,Supermarket
506585,INV-35909800006,04/19/2021,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14th St,Des Moines,50320,POINT (-93.596754 41.554101),77,POLK,1022200,100% Agave Tequila,55,SAZERAC NORTH AMERICA,988100,Monte Alban Silver,12,750.0,10.37,15.56,12,186.72,9.0,2.37,Supermarket
670913,INV-37056600007,06/01/2021,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14th St,Des Moines,50320,POINT (-93.596754 41.554101),77,POLK,1022200,100% Agave Tequila,55,SAZERAC NORTH AMERICA,988100,Monte Alban Silver,12,750.0,10.37,15.56,12,186.72,9.0,2.37,Supermarket


In [51]:
hv.loc[(hv["Item Description"] == "Monte Alban Silver") & (hv["Vendor Name"].isna()), "Vendor Name"] = "SAZERAC COMPANY INC"
hv.loc[(hv["Item Description"] == "Monte Alban Silver") & (hv["Vendor Number"].isna()), "Vendor Number"] = 55

In [52]:
hv[hv['Vendor Number'].isna()]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category


### INV-30475900020

In [53]:
hv[hv['Pack'].isna()]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
1745324,INV-30475900020,09/23/2020,2555,Hy-Vee Food Store / Keokuk,3111 Main,Keokuk,52632,POINT (-91.403752 40.41538),56,LEE,1041100,American Dry Gins,434,LUXCO INC,31658,Paramoun,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,Food Store


In [54]:
#checking who sells other products with that item number
hv[hv['Item Number'] == 31658].head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
32,INV-24112400010,12/23/2019,2568,Hy-Vee Food Store #4 / Cedar Rapids,1556 First Avenue NE,Cedar Rapids,52402,POINT (-91.650755 41.988967),57,LINN,1041100,American Dry Gins,434,LUXCO INC,31658,Paramount Gin,6,1750.0,7.11,10.67,6,64.02,10.5,2.77,Food Store
135,INV-24073800087,12/20/2019,2524,Hy-Vee Food Store / Dubuque,3500 Dodge St,Dubuque,52001,None,31,DUBUQUE,1041100,American Dry Gins,434,LUXCO INC,31658,Paramount Gin,6,1750.0,7.11,10.67,6,64.02,10.5,2.77,Food Store
645,INV-24070400065,12/20/2019,2569,Hy-Vee Food Store #2 / Cedar Rapids,279 Collins Road NE,Cedar Rapids,52402,POINT (-91.630677 42.027805),57,LINN,1041100,American Dry Gins,434,LUXCO INC,31658,Paramount Gin,6,1750.0,7.11,10.67,6,64.02,10.5,2.77,Food Store
1276,INV-24109000012,12/23/2019,2572,Hy-Vee Food Store / Cedar Falls,6301 University,Cedar Falls,50613,POINT (-92.435236 42.512789),7,BLACK HAWK,1041100,American Dry Gins,434,LUXCO INC,31658,Paramount Gin,6,1750.0,7.11,10.67,6,64.02,10.5,2.77,Food Store
3233,INV-24066600026,12/20/2019,2553,Hy-Vee Food Store / Creston,600 Sheldon,Creston,50801,POINT (-94.372552 41.051462),88,UNION,1041100,American Dry Gins,434,LUXCO INC,31658,Paramount Gin,6,1750.0,7.11,10.67,6,64.02,10.5,2.77,Food Store


In [55]:
hv.loc[(hv["Item Description"] == "Paramoun") & (hv["Pack"].isna()), "Pack"] = 6
hv.loc[(hv["Item Description"] == "Paramoun") & (hv["Bottle Volume (ml)"].isna()), "Bottle Volume (ml)"] = 1750.0
hv.loc[(hv["Item Description"] == "Paramoun") & (hv["State Bottle Cost"].isna()), "State Bottle Cost"] = 7.11
hv.loc[(hv["Item Description"] == "Paramoun") & (hv["State Bottle Retail"].isna()), "State Bottle Retail"] = 10.67
hv.loc[(hv["Item Description"] == "Paramoun") & (hv["Bottles Sold"].isna()), "Bottles Sold"] = 6
hv.loc[(hv["Item Description"] == "Paramoun") & (hv["Sale (Dollars)"].isna()), "Sale (Dollars)"] = 64.02
hv.loc[(hv["Item Description"] == "Paramoun") & (hv["Volume Sold (Liters)"].isna()), "Volume Sold (Liters)"] = 10.5
hv.loc[(hv["Item Description"] == "Paramoun") & (hv["Volume Sold (Gallons)"].isna()), "Volume Sold (Gallons)"] = 2.77

### Address

In [56]:
null_counts = hv.isnull().sum()
null_counts

Invoice/Item Number           0
Date                          0
Store Number                  0
Store Name                    0
Address                    1503
City                       1503
Zip Code                   1503
Store Location           266054
County Number              1503
County                     1503
Category                      0
Category Name                 0
Vendor Number                 0
Vendor Name                   0
Item Number                   0
Item Description              0
Pack                          0
Bottle Volume (ml)            0
State Bottle Cost             0
State Bottle Retail           0
Bottles Sold                  0
Sale (Dollars)                0
Volume Sold (Liters)          0
Volume Sold (Gallons)         0
Store Category                0
dtype: int64

#### Algona

In [57]:
hv[hv['Address'].isna()].head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
2478,INV-24164900115,12/26/2019,2585,Hy-Vee Wine and Spirits / Algona,None,None,<NA>,None,<NA>,None,1081400,American Schnapps,65,Jim Beam Brands,82780,Dekuyper Buttershots PET,12,750.0,6.50,9.75,12,117.00,9.0,2.37,Wine & Spirits
4294,INV-24164900174,12/26/2019,2585,Hy-Vee Wine and Spirits / Algona,None,None,<NA>,None,<NA>,None,1011400,Tennessee Whiskies,85,Brown Forman Corp.,26589,Gentleman Jack,6,1750.0,35.24,52.86,2,105.72,3.5,0.92,Wine & Spirits
4297,INV-24164900172,12/26/2019,2585,Hy-Vee Wine and Spirits / Algona,None,None,<NA>,None,<NA>,None,1052100,Imported Brandies,389,REMY COINTREAU USA INC,49186,Remy Martin VSOP,12,750.0,23.98,35.97,6,215.82,4.5,1.18,Wine & Spirits
4323,INV-24164900204,12/26/2019,2585,Hy-Vee Wine and Spirits / Algona,None,None,<NA>,None,<NA>,None,1051100,American Brandies,434,LUXCO INC,54056,Arrow Apricot Brandy,12,750.0,5.38,8.07,6,48.42,4.5,1.18,Wine & Spirits
4330,INV-24164900169,12/26/2019,2585,Hy-Vee Wine and Spirits / Algona,None,None,<NA>,None,<NA>,None,1052100,Imported Brandies,389,REMY COINTREAU USA INC,50686,St Remy VSOP Napoleon Brandy,12,750.0,7.24,10.86,6,65.16,4.5,1.18,Wine & Spirits


In [58]:
hv[(hv['Store Name'].str.contains('Algona', regex=False)) & (hv["Address"].notna())].head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
25297,INV-24300600053,01/02/2020,2585,Hy-Vee Wine and Spirits / Algona,1502 Highway 169 N Suite 1,Algona,50511,POINT (-94.235955 43.08091),55,KOSSUTH,1031200,American Flavored Vodka,259,Heaven Hill Brands,39922,Deep Eddy Peach,12,750.0,8.50,12.75,6,76.50,4.5,1.18,Wine & Spirits
25332,INV-24300600056,01/02/2020,2585,Hy-Vee Wine and Spirits / Algona,1502 Highway 169 N Suite 1,Algona,50511,POINT (-94.235955 43.08091),55,KOSSUTH,1081300,American Cordials & Liqueur,259,Heaven Hill Brands,72722,Christian Bros Honey,12,750.0,6.00,9.00,6,54.00,4.5,1.18,Wine & Spirits
25336,INV-24300600070,01/02/2020,2585,Hy-Vee Wine and Spirits / Algona,1502 Highway 169 N Suite 1,Algona,50511,POINT (-94.235955 43.08091),55,KOSSUTH,1012100,Canadian Whiskies,380,Phillips Beverage,14484,Revel Stoke Roasted Apple,12,750.0,8.34,12.51,6,71.94,4.5,1.18,Wine & Spirits
25710,INV-24300600014,01/02/2020,2585,Hy-Vee Wine and Spirits / Algona,1502 Highway 169 N Suite 1,Algona,50511,POINT (-94.235955 43.08091),55,KOSSUTH,1012100,Canadian Whiskies,421,SAZERAC COMPANY INC,11346,Seagrams VO Canadian Whiskey,12,750.0,5.23,7.85,12,94.20,9.0,2.37,Wine & Spirits
25728,INV-24300600093,01/02/2020,2585,Hy-Vee Wine and Spirits / Algona,1502 Highway 169 N Suite 1,Algona,50511,POINT (-94.235955 43.08091),55,KOSSUTH,1081400,American Schnapps,380,Phillips Beverage,84617,Phillips Root Beer Schnapps,12,1000.0,5.50,8.25,6,49.50,6.0,1.58,Wine & Spirits


In [ ]:
missing_store_address = hv[hv["Address"].isna()]["Store Name"].dropna().unique()

In [59]:
missing_store_address = hv[hv["Address"].isna()]["Store Name"].dropna().unique()
missing_store_city = hv[hv["City"].isna()]["Store Name"].dropna().unique()
missing_store_zip = hv[hv["Zip Code"].isna()]["Store Name"].dropna().unique()
missing_store_location = hv[hv["Store Location"].isna()]["Store Name"].dropna().unique()
missing_store_county_num = hv[hv["County Number"].isna()]["Store Name"].dropna().unique()
missing_store_county = hv[hv["County"].isna()]["Store Name"].dropna().unique()

In [60]:
missing_store_address

array(['Hy-Vee Wine and Spirits / Algona',
       'Hy-Vee Food Store / Mount Ayr',
       'Hy-Vee Food Store #2 / State Ankeny',
       'Hy-Vee Food Store / Corning', 'Hy-Vee Wine and Spirits / Lemars',
       'Hy-Vee Food Store / Centerville',
       'Hy-Vee Wine & Spirits #1 / MLK',
       'Hy-Vee Food and Drug #6 / Cedar Rapids',
       'Hy-Vee Food Store #1 / Ottumwa',
       'Hy-Vee Wine and Spirits / Estherville',
       'Hy-Vee Wine and Spirits / Atlantic',
       'Hy-Vee Drugstore / Council Bluffs'], dtype=object)

In [61]:
for store_name in missing_store_address:
    # Grab mode for `Address` using `Store Name`
    address_from_mode = hv.loc[(hv["Store Name"] == store_name)]["Address"].mode()

    # For now skipping none mode values
    if len(address_from_mode) == 0:
        continue
        
    # If it's not empty grab first value (mode)
    address_from_mode = address_from_mode[0]
    
    # Find where `Address` is missing ("N/A" or "None")
    #    Then replace with new value from mode
    hv.loc[hv['Store Name'] == store_name, 'Address'] = hv.loc[hv['Store Name'] == store_name, 'Address'].fillna(address_from_mode)

    
for store_name in missing_store_city:
    # Grab mode for `City` using `Store Name`
    city_from_mode = hv.loc[(hv["Store Name"] == store_name)]["City"].mode()

    # For now skipping none mode values
    if len(city_from_mode) == 0:
        continue
        
    # If it's not empty grab first value (mode)
    city_from_mode = city_from_mode[0]
    
    # Find where `City` is missing ("N/A" or "None")
    #    Then replace with new `Category` from mode
    hv.loc[hv['Store Name'] == store_name, 'City'] = hv.loc[hv['Store Name'] == store_name, 'City'].fillna(city_from_mode)

    
for store_name in missing_store_zip:
    # Grab mode for `Address` using `Store Name`
    zip_from_mode = hv.loc[(hv["Store Name"] == store_name)]["Zip Code"].mode()

    # For now skipping none mode values
    if len(zip_from_mode) == 0:
        continue
        
    # If it's not empty grab first value (mode)
    zip_from_mode = zip_from_mode[0]
    
    # Find where `Address` is missing ("N/A" or "None")
    #    Then replace with new `Category` from mode
    hv.loc[hv['Store Name'] == store_name, 'Zip Code'] = hv.loc[hv['Store Name'] == store_name, 'Zip Code'].fillna(zip_from_mode)

    
for store_name in missing_store_location:
    # Grab mode for `Address` using `Store Name`
    location_from_mode = hv.loc[(hv["Store Name"] == store_name)]["Store Location"].mode()

    # For now skipping none mode values
    if len(location_from_mode) == 0:
        continue
        
    # If it's not empty grab first value (mode)
    location_from_mode = location_from_mode[0]
    
    # Find where `Address` is missing ("N/A" or "None")
    #    Then replace with new `Category` from mode
    hv.loc[hv['Store Name'] == store_name, 'Store Location'] = hv.loc[hv['Store Name'] == store_name, 'Store Location'].fillna(location_from_mode)

    
for store_name in missing_store_county_num:
    # Grab mode for `Address` using `Store Name`
    county_num_from_mode = hv.loc[(hv["Store Name"] == store_name)]["County Number"].mode()

    # For now skipping none mode values
    if len(county_num_from_mode) == 0:
        continue
        
    # If it's not empty grab first value (mode)
    county_num_from_mode = county_num_from_mode[0]
    
    # Find where `Address` is missing ("N/A" or "None")
    #    Then replace with new `Category` from mode
    hv.loc[hv['Store Name'] == store_name, 'County Number'] = hv.loc[hv['Store Name'] == store_name, 'County Number'].fillna(county_num_from_mode)

    
for store_name in missing_store_county:
    # Grab mode for `Address` using `Store Name`
    county_from_mode = hv.loc[(hv["Store Name"] == store_name)]["County"].mode()

    # For now skipping none mode values
    if len(county_from_mode) == 0:
        continue
        
    # If it's not empty grab first value (mode)
    county_from_mode = county_from_mode[0]
    
    # Find where `Address` is missing ("N/A" or "None")
    #    Then replace with new `Category` from mode
    hv.loc[hv['Store Name'] == store_name, 'County'] = hv.loc[hv['Store Name'] == store_name, 'County'].fillna(county_from_mode)

In [62]:
null_counts = hv.isnull().sum()
null_counts

Invoice/Item Number           0
Date                          0
Store Number                  0
Store Name                    0
Address                       0
City                          0
Zip Code                      0
Store Location           174515
County Number                 0
County                        0
Category                      0
Category Name                 0
Vendor Number                 0
Vendor Name                   0
Item Number                   0
Item Description              0
Pack                          0
Bottle Volume (ml)            0
State Bottle Cost             0
State Bottle Retail           0
Bottles Sold                  0
Sale (Dollars)                0
Volume Sold (Liters)          0
Volume Sold (Gallons)         0
Store Category                0
dtype: int64

In [63]:
hv[hv['Store Location'].isna()]["Store Name"].unique()

array(['Hy-Vee Food Store / Indianola', 'Hy-Vee Food Store / Marion',
       'Hy-Vee Food Store / Chariton', 'Hy-Vee Food Store / Knoxville',
       'Hy-Vee Food Store / Dubuque', 'Hy-Vee Food Store #1 / WDM',
       'Hy-Vee / Windsor Heights', 'Hy-Vee Wine and Spirits / Waterloo',
       'Hy-Vee Wine & Spirits #2 / Davenport',
       'Hy-Vee Food Store #2 / Iowa City',
       'Hy-Vee Wine and Spirits / Spirit Lake',
       'Hy-Vee Fast & Fresh/Altoona', 'Hy-Vee C-Store / Fairfeild',
       'Hy-Vee Dollar Fresh / Maquoketa',
       'Hy-Vee Fast and Fresh / Grimes', 'Hy-Vee Gas / Johnston',
       'Hy-Vee C-Store / Fairfield', 'Hy-Vee Gas - Pleasant Hill',
       'Hy-Vee Wine and Spirits / Esthervill',
       'Hy-Vee Wine and Spirits #2', 'Hy-Vee Food Store / iowa Falls',
       'Hy-Vee Wine and Spirits / Spirit Lak',
       'Hy-Vee / Regal Liquors and Video'], dtype=object)

In [64]:
hv[hv['Store Location'].isna()].head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
8,INV-24080300084,12/20/2019,2549,Hy-Vee Food Store / Indianola,910 N Jefferson,Indianola,50125,None,91,WARREN,1022200,100% Agave Tequila,410,Patron Spirits Company,88291,Patron Silver Mini,10,50.0,20.30,30.45,1,30.45,0.05,0.01,Food Store
10,INV-24069700078,12/20/2019,2560,Hy-Vee Food Store / Marion,3600 Business Hwy 151 East,Marion,52302,None,57,LINN,1022200,100% Agave Tequila,619,Campari America,87250,Cabo Wabo Blanco,6,750.0,19.00,28.50,6,171.00,4.50,1.18,Food Store
11,INV-24081400061,12/20/2019,2551,Hy-Vee Food Store / Chariton,2001 West Court,Chariton,50049,None,59,LUCAS,1012100,Canadian Whiskies,259,Heaven Hill Brands,11786,Black Velvet PET,12,750.0,5.23,7.85,12,94.20,9.00,2.37,Food Store
29,INV-24083700038,12/20/2019,2566,Hy-Vee Food Store / Knoxville,813 N Lincoln Ste 1,Knoxville,50138,None,63,MARION,1081200,Cream Liqueurs,330,Gemini Spirits,80457,Ryans Cream Liqueur,12,1000.0,7.23,10.85,12,130.20,12.00,3.17,Food Store
31,INV-24080300010,12/20/2019,2549,Hy-Vee Food Store / Indianola,910 N Jefferson,Indianola,50125,None,91,WARREN,1011100,Blended Whiskies,65,Jim Beam Brands,27544,Red Stag Black Cherry,12,750.0,10.50,15.75,24,378.00,18.00,4.75,Food Store


In [65]:
hv[(hv['Store Name'].str.contains('Indianola', regex=False))]["Store Location"].mode()

0    POINT (-93.558417 41.36897)
dtype: object

In [66]:
hv[(hv['Store Name'].str.contains('Knoxville', regex=False)) & (hv["Store Location"].notna())].head(3)

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
236501,INV-33328700002,01/11/2021,2690,Hy-Vee Fast & Fresh - Knoxville,809 North Lincoln Street,Knoxville,50138,POINT (-93.109494 41.325392),63,MARION,1031100,American Vodkas,300,McCormick Distilling Co.,36904,McCormick 80prf Vodka PET,24,375.0,1.8,2.7,144,388.8,54.00,14.26,Fast & Fresh
237181,INV-33328700010,01/11/2021,2690,Hy-Vee Fast & Fresh - Knoxville,809 North Lincoln Street,Knoxville,50138,POINT (-93.109494 41.325392),63,MARION,1011400,Tennessee Whiskies,85,Brown Forman Corp.,26822,Jack Daniels Old #7 Black Label,48,100.0,2.0,3.0,12,36.0,1.20,0.31,Fast & Fresh
237476,INV-33328700005,01/11/2021,2690,Hy-Vee Fast & Fresh - Knoxville,809 North Lincoln Street,Knoxville,50138,POINT (-93.109494 41.325392),63,MARION,1062400,Spiced Rum,260,DIAGEO AMERICAS,43331,Captain Morgan Original Spiced Mini,12,50.0,4.8,7.2,3,21.6,0.15,0.03,Fast & Fresh


In [67]:
hv[(hv['Store Name'].str.contains('Indianola', regex=False)) & (hv["Store Location"].notna())].head(3)

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Store Category
133299,INV-32765100003,12/16/2020,2676,Hy-Vee Gas / Indianola,912 N Jefferson Way,Indianola,50125,POINT (-93.558417 41.36897),91,WARREN,1031100,American Vodkas,301,FIFTH GENERATION INC,38194,Titos Handmade Vodka Mini,5,50.0,12.80,19.20,5,96.00,0.25,0.06,Gas
133896,INV-32765100002,12/16/2020,2676,Hy-Vee Gas / Indianola,912 N Jefferson Way,Indianola,50125,POINT (-93.558417 41.36897),91,WARREN,1031100,American Vodkas,301,FIFTH GENERATION INC,38174,Titos Handmade Vodka,12,375.0,5.42,8.13,12,97.56,4.50,1.18,Gas
134952,INV-32765100004,12/16/2020,2676,Hy-Vee Gas / Indianola,912 N Jefferson Way,Indianola,50125,POINT (-93.558417 41.36897),91,WARREN,1081600,Whiskey Liqueur,421,SAZERAC COMPANY INC,64858,Fireball Cinnamon Whiskey Mini Dispenser,1,50.0,25.80,38.70,3,135.00,0.15,0.03,Gas


- unable to fill the remaining 174515 values as previous records do not exist
- Since missing rows only make up 0.7% of the data, will drop

In [68]:
hv = hv.dropna()

In [69]:
null_counts = hv.isnull().sum()
null_counts

Invoice/Item Number      0
Date                     0
Store Number             0
Store Name               0
Address                  0
City                     0
Zip Code                 0
Store Location           0
County Number            0
County                   0
Category                 0
Category Name            0
Vendor Number            0
Vendor Name              0
Item Number              0
Item Description         0
Pack                     0
Bottle Volume (ml)       0
State Bottle Cost        0
State Bottle Retail      0
Bottles Sold             0
Sale (Dollars)           0
Volume Sold (Liters)     0
Volume Sold (Gallons)    0
Store Category           0
dtype: int64

In [70]:
hv.to_csv("hv_cleaned.csv")

In [76]:
len(hv['Zip Code'].unique())

112

In [74]:
len(hv['Category Name'].unique())

75